In [1]:
import tensorflow as tf

import keras
from keras.losses import cosine_similarity

from models import TextFeatureExtractorLayer, GloveEmbeddingLayer
from utils import GoogleRestaurantsReviewDataset

## Load the Train and Test Dataset

In [2]:
max_seq_length = 500
dataset = GoogleRestaurantsReviewDataset(max_seq_length=max_seq_length)
text_vectorize = dataset.text_vectorize

print('Load training data')
train_X_user, train_X_bus, train_y = dataset.load_train_or_test_dataset(train=True)
print(f'Total {train_y.shape} training data\n')

print('Load test data')
test_X_user, test_X_bus, test_y = dataset.load_train_or_test_dataset(train=False)
print(f'Total {test_y.shape} test data\n')

Build Training data


78422it [01:50, 712.47it/s]


Build Testing data


19606it [00:15, 1298.85it/s]


Load training data
Total (78422,) training data

Load test data
Total (19606,) test data



## Build the Model

In [3]:
print('Build model')
embedding = GloveEmbeddingLayer(num_tokens=len(text_vectorize),
                                vocabulary_dict=text_vectorize.vocabulary)

user_inputs = keras.Input(shape=(None,), dtype="int64")
x = embedding(user_inputs)
user_outputs = TextFeatureExtractorLayer(
    input_dim=(dataset.max_seq_length, embedding.embed_dim), output_dim=64)(x)

bus_inputs = keras.Input(shape=(None,), dtype="int64")
y = embedding(bus_inputs)
bus_outputs = TextFeatureExtractorLayer(
    input_dim=(dataset.max_seq_length, embedding.embed_dim), output_dim=64)(y)

outputs = -cosine_similarity(user_outputs, bus_outputs, axis=1)

model = keras.Model([user_inputs, bus_inputs], outputs)
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

Build model
Metal device set to: Apple M1 Max


2023-04-23 12:04:19.956369: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-23 12:04:19.956510: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 glove_embedding_layer (GloveEm  (None, None, 300)   10695900    ['input_1[0][0]',                
 beddingLayer)                                                    'input_3[0][0]']                
                                                                                                  
 text_feature_extractor_layer (  (None, 64)          2890012     ['glove_embedding_layer[0][0]

### Load Trained Params

In [4]:
checkpoint_path = "./training_2/cp.ckpt"
model.load_weights(checkpoint_path)

## Predict

In [5]:
num_samples = 100
predicted = model.predict([test_X_user[:num_samples], test_X_bus[:num_samples]])
for i in range(num_samples):
    print('predicted: %.4f, actual: %.4f' % (predicted[i], test_y[i]))

2023-04-23 12:04:25.183278: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-23 12:04:25.804237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 9s 1s/step
predicted: 0.0000, actual: -1.0000
predicted: 0.6140, actual: 1.0000
predicted: 0.0139, actual: -1.0000
predicted: 0.2976, actual: 1.0000
predicted: 0.0000, actual: -1.0000
predicted: 0.3293, actual: -1.0000
predicted: 0.6051, actual: 1.0000
predicted: 0.5342, actual: 1.0000
predicted: 0.2366, actual: -1.0000
predicted: 0.0000, actual: 1.0000
predicted: 0.3357, actual: -1.0000
predicted: 0.0000, actual: -1.0000
predicted: 0.0000, actual: -1.0000
predicted: 0.0000, actual: 1.0000
predicted: 0.7817, actual: 1.0000
predicted: 0.7039, actual: 1.0000
predicted: 0.1982, actual: 1.0000
predicted: 0.0000, actual: 1.0000
predicted: 0.3506, actual: -1.0000
predicted: 0.0000, actual: 1.0000
predicted: 0.6307, actual: -1.0000
predicted: 0.2608, actual: -1.0000
predicted: 0.5730, actual: -1.0000
predicted: 0.0000, actual: -1.0000
predicted: 0.7568, actual: 1.0000
predicted: 0.7010, actual: 1.0000
predicted: 0.8220, actual: -1.0000
predicted: 0.0744,

# Evaluation

## Evaluating on Training Data

In [6]:
from tqdm import tqdm
import numpy as np

In [28]:
model.predict([train_X_user[0:1], train_X_bus[0:1]])[0]

1/1 [==============================] - 0s 83ms/step


1.23896934e-11

In [40]:
predict = model.predict([np.vstack([[train_X_user[3]]*10000]), train_X_bus[:10000]])

313/313 [==============================] - 32s 103ms/step


In [41]:
np.sort(predict)

array([0.58466494, 0.58466494, 0.58466494, ..., 0.58466494, 0.58466494,
       0.58466494], dtype=float32)